In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors45678

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

Solving environment: done

# All requested packages already installed.



Data is loaded, preprocessed and stored in file.

In [2]:
df=pd.read_csv('data_toronto.csv')
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [3]:
df.groupby('Borough').count()

,Postal Code,Neighborhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,19,19,19,19
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,12,12,12,12
Mississauga,1,1,1,1
North York,24,24,24,24
Scarborough,17,17,17,17
West Toronto,6,6,6,6


North York has highest number of neighborhood while Mississauga has least.

In [4]:
def get_location(address):
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude,longitude


In [5]:
CLIENT_ID = 'VT5DTAMJ3XFEBH5LMV441H5FYK4DWQ4IJ1RQ2H5RNP1FHLQY' 
CLIENT_SECRET = 'EQEYMPHSHBO1ADOZVKKDHRLN4HFW2P2BKKF1S24AZSTDMNLY' 
VERSION = '20180605' 

In [6]:
def get_neighborhood(borough):
    df_neigh=df[df['Borough']==borough]
    return df_neigh

Split data and make dataframe for each borough

In [7]:
north_york=get_neighborhood('North York').reset_index()

In [8]:
downtown_toronto=get_neighborhood('Downtown Toronto').reset_index()

In [9]:
scarborough=get_neighborhood('Scarborough').reset_index()

In [10]:
etobicoke=get_neighborhood('Etobicoke').reset_index()

In [11]:
central_toronto=get_neighborhood('Central Toronto').reset_index()

In [12]:
west_toronto=get_neighborhood('West Toronto').reset_index()

In [13]:
east_toronto=get_neighborhood('East Toronto').reset_index()

In [14]:
east_york=get_neighborhood('East York').reset_index()

In [15]:
york=get_neighborhood('York').reset_index()

In [16]:
mississauga=get_neighborhood('Mississauga').reset_index()

In [17]:
def get_loc(addr):
    lat=addr.loc[0,'Latitude']
    lng=addr.loc[0,'Longitude']
    name=addr.loc[0,'Neighborhood']
    return lat,lng,name

In [18]:

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

Let's create a function to find nearby venues in all the neighborhood. From the previous module, we know that all the information is in the **items** key.

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    venue_cat=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
                   
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        for v in results:
            try:  
                v_name=v['venue']['name'], 
                v_lat=v['venue']['location']['lat'], 
                v_lng=v['venue']['location']['lng'],  
                v_cat=v['venue']['categories'][0]['name']
                venue_cat.append(v_cat)
                venues_list.append([name,lat,lng,v_name,v_lat,v_lng,v_cat])
            except KeyError:
                pass
        
        column = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
        nearby_venues=pd.DataFrame(venues_list,columns=column)
              
    return nearby_venues,venue_cat

Now get nearby venues in the form of dataframe and venues category for all neighborhoods in each borough

In [20]:
north_york_venues,north_york_cat = getNearbyVenues(names=north_york['Neighborhood'],
                                   latitudes=north_york['Latitude'],
                                   longitudes=north_york['Longitude']
                                  )

In [21]:
downtown_toronto_venues,downtown_toronto_cat = getNearbyVenues(names=downtown_toronto['Neighborhood'],
                                   latitudes=downtown_toronto['Latitude'],
                                   longitudes=downtown_toronto['Longitude']
                                  )

In [22]:
scarborough_venues,scarborough_cat = getNearbyVenues(names=scarborough['Neighborhood'],
                                   latitudes=scarborough['Latitude'],
                                   longitudes=scarborough['Longitude']
                                  )

In [23]:
etobicoke_venues,etobicoke_cat = getNearbyVenues(names=etobicoke['Neighborhood'],
                                   latitudes=etobicoke['Latitude'],
                                   longitudes=etobicoke['Longitude']
                                  )

In [24]:
central_toronto_venues,central_toronto_cat = getNearbyVenues(names=central_toronto['Neighborhood'],
                                   latitudes=central_toronto['Latitude'],
                                   longitudes=central_toronto['Longitude']
                                  )

In [25]:
west_toronto_venues,west_toronto_cat = getNearbyVenues(names=west_toronto['Neighborhood'],
                                   latitudes=west_toronto['Latitude'],
                                   longitudes=west_toronto['Longitude']
                                  )

In [26]:
east_toronto_venues,east_toronto_cat = getNearbyVenues(names=east_toronto['Neighborhood'],
                                   latitudes=east_toronto['Latitude'],
                                   longitudes=east_toronto['Longitude']
                                  )

In [27]:
east_york_venues,east_york_cat = getNearbyVenues(names=east_york['Neighborhood'],
                                   latitudes=east_york['Latitude'],
                                   longitudes=east_york['Longitude']
                                  )

In [28]:
mississauga_venues,mississauga_cat = getNearbyVenues(names=mississauga['Neighborhood'],
                                   latitudes=mississauga['Latitude'],
                                   longitudes=mississauga['Longitude']
                                  )

In [29]:
york_venues,york_cat = getNearbyVenues(names=york['Neighborhood'],
                                   latitudes=york['Latitude'],
                                   longitudes=york['Longitude']
                                  )

Find all categories of venue in the city of Toronto.

In [30]:
all_venue=set(mississauga_cat+york_cat+east_york_cat+east_toronto_cat+west_toronto_cat+central_toronto_cat+etobicoke_cat+scarborough_cat+downtown_toronto_cat+north_york_cat)
len(all_venue)

272

Define a function and find the number of venues not present in boroughs

In [31]:
def Diff(li1, li2): 
    return (list(set(li1) - set(li2))) 

In [32]:
north_york_n=Diff(all_venue,north_york_cat)
len(north_york_n)

167

In [33]:
york_n=Diff(all_venue,york_cat)
len(york_n)

254

In [34]:
mississauga_n=Diff(all_venue,mississauga_cat)
len(mississauga_n)

261

In [35]:
east_york_n=Diff(all_venue,east_york_cat)
len(east_york_n)

226

In [36]:
east_toronto_n=Diff(all_venue,east_toronto_cat)
len(east_toronto_n)

205

In [37]:
west_toronto_n=Diff(all_venue,west_toronto_cat)
len(west_toronto_n)

193

In [38]:
central_toronto_n=Diff(all_venue,central_toronto_cat)
len(west_toronto_n)

193

In [39]:
downtown_toronto_n=Diff(all_venue,downtown_toronto_cat)
len(downtown_toronto_n)

63

In [40]:
etobicoke_n=Diff(all_venue,etobicoke_cat)
len(etobicoke_n)

233

In [41]:
scarborough_n=Diff(all_venue,scarborough_cat)
len(scarborough_n)

215

In [42]:
def sum_diff(a1,a2,a3,a4,a5,a6,a7,a8):
    return (set(a1).intersection(set(a2),set(a3),set(a4),set(a5),set(a6),set(a7),set(a8)))

In [43]:
s=sum_diff(north_york_n,downtown_toronto_n,east_york_n,east_toronto_n,west_toronto_n,central_toronto_n,etobicoke_n,scarborough_n)


In [44]:
s

{'Field'}

This results shows the places not present in most of boroughs.

####  Analyze Each Neighborhood

In [45]:
def analyze_neighborhood(neighborhood):
    neighborhood_en = pd.get_dummies(neighborhood[['Venue Category']], prefix="", prefix_sep="")
    neighborhood_en['Neighborhood'] = neighborhood['Neighborhood'] 
    neighborhood_en=neighborhood_en.set_index('Neighborhood').reset_index()# add neighborhood column back to dataframe
    neighborhood_en_grp = neighborhood_en.groupby('Neighborhood').mean().reset_index()
    
    #get top 5 most common venues
    num_top_venues = 5

    for hood in neighborhood_en_grp['Neighborhood']:
        print("----"+hood+"----")
        temp = neighborhood_en_grp[neighborhood_en_grp['Neighborhood'] == hood].T.reset_index()
        temp.columns = ['venue','freq']
        temp = temp.iloc[1:]
        temp['freq'] = temp['freq'].astype(float)
        temp = temp.round({'freq': 2})
        print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
        print('\n')

Let's print each neighborhood along with the top 5 most common venues. Common venues will help in deciding market and trade.

In [46]:
#Mississauga has only 1 neighborhood
analyze_neighborhood(mississauga_venues)

----Canada Post Gateway Processing Centre----
                 venue  freq
0          Coffee Shop  0.15
1                Hotel  0.15
2  American Restaurant  0.08
3        Burrito Place  0.08
4  Fried Chicken Joint  0.08




Since Mississauga has only 1 neighborhood, it has potential for large scale development. 

In [47]:
analyze_neighborhood(york_venues)

----Caledonia-Fairbanks----
           venue  freq
0            Bar   0.2
1           Park   0.2
2           Pool   0.2
3         Market   0.2
4  Women's Store   0.2


----Del Ray, Mount Dennis, Keelsdale and Silverthorn----
                  venue  freq
0  Caribbean Restaurant  0.17
1          Skating Rink  0.17
2  Fast Food Restaurant  0.17
3        Sandwich Place  0.17
4   Fried Chicken Joint  0.17


----Humewood-Cedarvale----
          venue  freq
0         Trail  0.25
1       Dog Run  0.25
2         Field  0.25
3  Hockey Arena  0.25
4           Bar  0.00


----Runnymede, The Junction North----
                  venue  freq
0  Caribbean Restaurant  0.25
1     Convenience Store  0.25
2           Pizza Place  0.25
3        Breakfast Spot  0.25
4                   Bar  0.00


----Weston----
            venue  freq
0            Park   1.0
1             Bar   0.0
2  Breakfast Spot   0.0
3           Trail   0.0
4    Skating Rink   0.0




York has 5 neighborhood with 0 frequency of most spots like Bar or any kind of restaurants.

In [48]:
analyze_neighborhood(downtown_toronto_venues)

----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2          Café  0.04
3      Beer Bar  0.04
4   Cheese Shop  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3   Harbor / Marina  0.06
4               Bar  0.06


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.16
1   Italian Restaurant  0.06
2  Japanese Restaurant  0.06
3       Sandwich Place  0.06
4                 Café  0.04


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12
3     Baby Store  0.06
4    Candy Store  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.08
1     Sushi Restaurant  0.06
2  Japanese Restaurant  0.05
3           Restaurant  0.04
4              Gay Bar  0.04


----Comm

Downtown Toronto has many commercial and entertainment sites. It can be good spot for apartment or daily wage workers.

Suppose a person want to look for an apartment or restaurant or gym near university of Toronto, then he/she can search for that.

In [49]:
search_query = 'gym'
radius = 500
LIMIT=100
name='University of toronto'
lat,lng=get_location('University of toronto')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, search_query, radius, LIMIT)
venues_list=[]
results = requests.get(url).json()
venues = results['response']['venues']

for v in venues:
        try:  
                v_name=v['name'].rstrip(',')
                v_addr=v['location']['address']
                v_lat=str(v['location']['lat']).rstrip(',') 
                v_lng=str(v['location']['lng']).rstrip(',')  
                v_cat=v['categories'][0]['name']
                #venue_cat.append(v_cat)
                venues_list.append([name,lat,lng,v_name,v_addr,v_lat,v_lng,v_cat])
        except KeyError:
                pass
        
        column = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue',
                  'Venue Address',
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
        search_result=pd.DataFrame(venues_list,columns=column)
search_result.head()            


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Address,Venue Latitude,Venue Longitude,Venue Category
0,University of toronto,43.663462,-79.39776,Innis College Residence Gym,111 St. George St,43.66573782314874,-79.3983588615895,Gym
1,University of toronto,43.663462,-79.39776,Hart House Gym,7 Hart House Cir.,43.66417202442517,-79.39488839857401,Gym
2,University of toronto,43.663462,-79.39776,UTS Gym,271 Bloor Street West,43.666427363551094,-79.40322947083064,High School


In [50]:
#let's visualize the search result
lat,lng=get_location('University of Toronto, Toronto')
map_m = folium.Map(location=[lat,lng], zoom_start=14)

# add markers to map
for lat, lng, label in zip(search_result['Venue Latitude'], search_result['Venue Longitude'], search_result['Venue']):
    
    label = folium.Popup(label, parse_html=True)
    folium.features.CircleMarker(
        [float(lat), float(lng)],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_m)  
    
map_m
